# Indexed cellular automation


In [1]:
from __future__ import annotations
from dataclasses import dataclass
from itertools import chain, cycle, islice, repeat
from typing import Final, Iterator

import numpy as np
from scipy.signal import convolve2d


KERNEL: Final[np.array] = 2 ** np.arange(9, dtype=np.uint16).reshape(3, 3)


@dataclass
class ImageEnhancer:
    image: np.array[bool]
    algorithm: np.array[bool]

    @classmethod
    def from_scanner(cls, lines: str) -> ImageEnhancer:
        algo, _, image = lines.partition("\n\n")

        return cls(
            np.array([[c == "#" for c in line] for line in image.splitlines()]),
            np.array([c == "#" for c in algo]),
        )

    @property
    def edge_fill(self) -> Iterator[bool]:
        blank, full = self.algorithm[[0, -1]]
        if not blank:
            return repeat(False)
        if not full:
            return cycle((False, True))
        return chain((False,), repeat(True))

    def enhance(self, steps: int) -> np.array[bool]:
        img, algo = self.image, self.algorithm
        for edge in islice(self.edge_fill, steps):
            img = algo[convolve2d(img, KERNEL, fillvalue=edge)]
        return img


test_image = ImageEnhancer.from_scanner(
    """\
..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##\
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###\
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.\
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....\
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..\
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....\
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.\n#....\n##..#\n..#..\n..###
"""
)

assert test_image.enhance(2).sum() == 35


In [2]:
import aocd

scanned_image = ImageEnhancer.from_scanner(aocd.get_data(day=20, year=2021))
print("Part 1:", scanned_image.enhance(2).sum())


Part 1: 5275


In [3]:
assert test_image.enhance(50).sum() == 3351


In [4]:
print("Part 2:", scanned_image.enhance(50).sum())


Part 2: 16482
